In [10]:
# data from https://github.com/unitedstates/congress
import glob, json, csv

people_party = {}
scrutins = []
for file in glob.glob("/home/damien/repos/congress/data/116/votes/2020/*/*.json"):
    scrutin = json.load(open(file))
    scrutins.append(scrutin)
    for position in 'Nay', 'Yea', 'Not Voting', 'Present':
        for vote in scrutin.get('votes', {}).get(position, []):
            people_party[vote['display_name']] = vote['party']

print(len(scrutins), 'scrutins')
print(len(people_party), 'people')

442 scrutins
538 people


In [22]:
def position(scrutin, people):
    for position in 'Nay', 'Yea', 'Not Voting', 'Present':
        for vote in scrutin.get('votes', {}).get(position, []):
            if vote['display_name'] == people:
                return position

def similarity(people1, people2):
    global scrutins
    total = 0
    total_common = 0
    for scrutin in scrutins:
        pos1 = position(scrutin, people1)
        pos2 = position(scrutin, people2)
        if not pos1 or not pos2:
            continue
        if pos1 == pos2:
            total_common += 1
        total += 1
    if total == 0:
        return 0, 0
    return total_common / total, total

print(similarity('Ocasio-Cortez', 'Omar'))

(0.9510869565217391, 184)


# Similar to AOC

In [23]:
sims = [[id, similarity(id, 'Ocasio-Cortez')] for id in people_party]
sims.sort(key=lambda x: -x[1][0])
for id, sim in sims:
    print(id, sim)

Ocasio-Cortez (1.0, 184)
Omar (0.9510869565217391, 184)
Pressley (0.9510869565217391, 184)
Tlaib (0.9510869565217391, 184)
Khanna (0.9456521739130435, 184)
Pocan (0.9456521739130435, 184)
Levin (MI) (0.9293478260869565, 184)
McGovern (0.9293478260869565, 184)
Meng (0.9293478260869565, 184)
Raskin (0.9293478260869565, 184)
Blumenauer (0.9239130434782609, 184)
Doyle, Michael F. (0.9239130434782609, 184)
Espaillat (0.9239130434782609, 184)
García (IL) (0.9239130434782609, 184)
Kildee (0.9239130434782609, 184)
Lee (CA) (0.9239130434782609, 184)
Lowenthal (0.9239130434782609, 184)
Takano (0.9239130434782609, 184)
Watson Coleman (0.9239130434782609, 184)
Chu, Judy (0.9184782608695652, 184)
Clark (MA) (0.9184782608695652, 184)
Dingell (0.9184782608695652, 184)
Jayapal (0.9184782608695652, 184)
Jeffries (0.9184782608695652, 184)
Maloney, Carolyn B. (0.9184782608695652, 184)
Neal (0.9184782608695652, 184)
Vargas (0.9184782608695652, 184)
Clarke (NY) (0.9130434782608695, 184)
Eshoo (0.9130434782

In [29]:
import pandas as pd
from bokeh.plotting import figure, output_file, show

AOC = 'Ocasio-Cortez'
other = 'Cheney'

data = []
for id, party in people_party.items():
    sim, total = similarity(AOC, id)
    sim_other, total_other = similarity(other, id)
    if total < 20 or total_other < 20:
        continue
    data.append({
        'sim_AOC': sim,
        'sim_Cheney': sim_other,
        'name': id,
        'party': party,
    })
df = pd.DataFrame(data)
import plotly.express as px
# fig = px.scatter_3d(df, x=df.similarité_ruffin, y=df.similarité_bergé, z=df.similarité_abad, color=partis, hover_name='nom')
fig = px.scatter(df, x=df.sim_AOC, y=df.sim_Cheney, color=df.party, hover_name=df.name)
fig.show()
fig.write_html("demo.html")